## Preliminaries

In [ ]:
# Global settings
options(scipen=999)

Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
# Paths
ROOT_PATH <- '~/workspace'
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration')
CODE_PATH <- file.path(ROOT_PATH, 'code')
DATA_PATH <- file.path(ROOT_PATH, 'data')
OUTPUT_DATA_PATH <- file.path(DATA_PATH, 'seasonality')

In [ ]:
# Load utils
source(file.path(CODE_PATH, "snt_utils.r"))

In [ ]:
# List required pcks
required_packages <- c(
  "jsonlite",
  "data.table",
  "ggplot2",
  "fpp3",
  "arrow",
  "glue",
  "sf",
  "RColorBrewer",
  "httr",
  "reticulate",
  "arrow"
)

# Execute function
install_and_load(required_packages)

In [ ]:
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

In [ ]:
# Load SNT config
CONFIG_FILE_NAME <- "SNT_config.json"
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) },
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

# Set config variables
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
era5_dataset <- config_json$SNT_DATASET_IDENTIFIERS$ERA5_DATASET_CLIMATE
dhis2_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

print(paste("Country code: ", COUNTRY_CODE))

## Parameters

In [ ]:
# Parameters
# type_of_seasonality <- 'cases' # 'precipitation' | 'cases'
# minimum_periods <- as.integer(48)
# maximum_proportion_missings_overall <- 0.1
# maximum_proportion_missings_per_district <- 0.2
# minimum_month_block_size <- as.integer(3)
# maximum_month_block_size <- as.integer(5)
# threshold_for_seasonality <- 0.6
# threshold_proportion_seasonal_years <- 0.5

In [ ]:
log_msg(glue("Running Seasonality Analysis : ({type_of_seasonality}):"))

**Fixed routine formatting columns**

In [ ]:
# Fixed routine formatting columns 
admin_id_col <- 'ADM2_ID'  
year_col <- 'YEAR'
month_col <- 'MONTH'
period_cols <- c(year_col, month_col)

In [ ]:
possible_month_block_sizes <- as.integer(minimum_month_block_size:maximum_month_block_size)
formatted_threshold_for_seasonality <- sprintf("%d%%", round(threshold_for_seasonality * 100))
print(paste("Formatted threshold :",formatted_threshold_for_seasonality))

## Load data

In [ ]:
# Load spatial file from dataset
spatial_data_filename <- paste(COUNTRY_CODE, "shapes.geojson", sep = "_")
spatial_data <- get_latest_dataset_file_in_memory(dhis2_dataset, spatial_data_filename)
log_msg(glue("File {spatial_data_filename} successfully loaded from dataset version: {dhis2_dataset}"))

**Load time series data**

In [ ]:
if (type_of_seasonality == "precipitation") {
    
    # Load precipitation data from dataset
    precipitation_data_filename <- paste(COUNTRY_CODE, "total_precipitation_monthly.parquet", sep = "_")
    original_dt <- get_latest_dataset_file_in_memory(era5_dataset, precipitation_data_filename)
    log_msg(glue("File {precipitation_data_filename} successfully loaded from dataset version: {era5_dataset}"))
    original_values_col <- 'MEAN'
    
} else if (type_of_seasonality == "cases") {
    
    # Load routine data from dataset
    routine_data_filename <- paste(COUNTRY_CODE, "routine.parquet", sep = "_")
    original_dt <- get_latest_dataset_file_in_memory(dhis2_dataset, routine_data_filename)
    log_msg(glue("File {routine_data_filename} successfully loaded from dataset version: {dhis2_dataset}"))
    original_values_col <- 'CONF'
    
} else {
    log_msg(glue("Seasonality type not supported : {type_of_seasonality}"), level="error")
    stop()
}

In [ ]:
# Columns formatting
admin_data <- st_drop_geometry(spatial_data)
setDT(admin_data)
common_cols <- names(admin_data)

seasonality_col <- glue('SEASONALITY', toupper(type_of_seasonality), .sep = "_")
season_duration_col <- glue('SEASONAL_BLOCK_DURATION', toupper(type_of_seasonality), .sep = "_")
final_table_cols <- c(names(admin_data), seasonality_col, season_duration_col)
print(final_table_cols)

**Create the containers for the data**

In [ ]:
# Create an empty table if the analysis is stopped for lack of enough data
seasonality_cols <- c(seasonality_col, season_duration_col)
empty_dt <- copy(admin_data)[, (seasonality_cols) := NA]

**Remove imputed files (if any)**

In [ ]:
# get list of files
files_in_folder <- list.files(OUTPUT_DATA_PATH, full.names = TRUE)
files_to_remove <- files_in_folder[grepl(paste0(COUNTRY_CODE, "_", type_of_seasonality, "_imputed"), basename(files_in_folder), ignore.case = TRUE)]

# remove
if (length(files_to_remove) > 0) {
  file.remove(files_to_remove)
}
print(glue("Deleted files: {files_to_remove}"))

## Preprocess input data

In [ ]:
# format table
setDT(original_dt)
integer_cols <- c(year_col, month_col)
numeric_cols <- c(original_values_col)
original_dt[, (integer_cols) := lapply(.SD, as.integer), .SDcols = integer_cols]
head(original_dt, 3)

In [ ]:
# keep only the useful columns and aggregate the data on them
original_dt <- original_dt[,
                           setNames(list(sum(get(original_values_col), na.rm = TRUE)), original_values_col), 
                           by = c(admin_id_col, period_cols)
                           ]

num_periods <- make_cartesian_admin_period(original_dt, admin_id_col, year_col, month_col)[[1]]
all_rows <- make_cartesian_admin_period(original_dt, admin_id_col, year_col, month_col)[[2]]

if (num_periods < minimum_periods){    
    log_msg(glue("Data is not reliable: 
                    at least {minimum_periods} year-month periods of data are required for the case analysis; 
                    the data only contains {num_periods} periods. Abandoning analysis.")
           , level="error")
    stop("ERROR 1")
}

# inject the (possibly missing) rows into the data
original_dt <- make_full_time_space_data(
  input_dt=original_dt,
  full_rows_dt=all_rows,
  target_colname=original_values_col,
  admin_colname=admin_id_col,
  year_colname=year_col,
  month_colname=month_col)

if(nrow(original_dt[is.na(get(original_values_col)),]) > (maximum_proportion_missings_overall * nrow(original_dt))){    
    log_msg("There are too many missing values in the data overall. Abandoning analysis.", level="error")
    stop("ERROR 2")   
}

### Imputation of missings

In [ ]:
# imputed filename
filename_imputed_csv <- paste(COUNTRY_CODE, type_of_seasonality, 'imputed.csv', sep = '_')
filename_imputed_parquet <- paste(COUNTRY_CODE, type_of_seasonality, 'imputed.parquet', sep = '_')

In [ ]:
# create the name of the column which will store the imputed/estimated values
imputed_col = paste(original_values_col, 'EST', sep = '_')

# if there are rows of missing data for cases, impute them (SARIMA)
if(nrow(original_dt[!is.na(get(original_values_col)),]) != nrow(original_dt)) {
    log_msg("There is missing data. Proceeding to impute them.", level="warning")
    
    # extract data on only the administrative units which have missing values for original_values_col
    missing_dt <- extract_dt_with_missings(original_dt, target_colname = original_values_col, id_colname = admin_id_col)
    missing_dt <- missing_dt[, PERIOD := make_yearmonth(year = YEAR, month = MONTH)]
    missing_dt <- missing_dt[, .SD, .SDcols = c(admin_id_col, 'PERIOD', original_values_col)]
    
    # how many rows missing for each administrative unit? if too many, then not good idea to impute
    missings_by_admin_unit <- missing_dt[, .(missing_count = sum(is.na(get(original_values_col)))), by = admin_id_col][order(-missing_count)]
    
    # if for any given admin unit, more than a given % of data is missing, there's too much to impute (maybe should be stricter - to discuss)
    if(missings_by_admin_unit[, max(missing_count)] > maximum_proportion_missings_per_district * num_periods){
      log_msg("Some administrative units have too many missing values in the target data. Abandoning analysis.", level="error")
      stop("ERROR 3")
    }
    
    # split to list per admin_unit_id, to apply SARIMA imputation on each time series (per admin unit)
    missing_districts_list <- split(missing_dt, by = admin_id_col)
    
    # seasonal ARIMA to estimate missing cases: apply function to list of data.tables with missing rows, then create data.table from result
    filled_missings_dt <- rbindlist(
    lapply(missing_districts_list,
           fill_missing_cases_ts,
           original_values_colname=original_values_col,
           estimated_values_colname=imputed_col,
           admin_colname=admin_id_col,
           period_colname='PERIOD',
           threshold_for_missing = 0.0)
    )
    
    # add the imputed ("_EST") values to the original data
    imputed_dt <- merge.data.table(original_dt, filled_missings_dt[, .SD, .SDcols = !(original_values_col)], by = c(admin_id_col, year_col, month_col), all.x = TRUE)
    
    # copy from the districts without missings;
    # if data is large, this could be made faster by only copying from the districts which are not in the missing_dt
    imputed_dt[!is.na(get(original_values_col)), (imputed_col) := get(original_values_col)]

    # Save imputed file, only if it was computed..
    fwrite(imputed_dt, file = file.path(OUTPUT_DATA_PATH, filename_imputed_csv))
    write_parquet(imputed_dt, file.path(OUTPUT_DATA_PATH, filename_imputed_parquet))
    log_msg(glue("Imputed file saved under: {file.path(OUTPUT_DATA_PATH, filename_imputed_csv)}"))
    
} else {    
    imputed_dt <- copy(original_dt)
    imputed_dt[, (imputed_col) := get(original_values_col)]
}

## Seasonality

In [ ]:
# The seasonality per row (period-admin unit) -----------------------------
log_msg("Computing month seasonality.")

row_seasonality_dt <- compute_month_seasonality(
  input_dt=imputed_dt,
  indicator=type_of_seasonality,
  values_colname=imputed_col,
  vector_of_durations=possible_month_block_sizes,
  admin_colname=admin_id_col,
  year_colname=year_col,
  month_colname=month_col,
  proportion_threshold=threshold_for_seasonality
)

# The seasonality per admin unit, irrespective of year ----------------------
log_msg("Computing seasonality per administrative unit.")

seasonality_source_dt <- process_seasonality(
  input_dt=row_seasonality_dt,
  indicator=type_of_seasonality,
  vector_of_durations=possible_month_block_sizes,
  admin_colname=admin_id_col,
  year_colname=year_col,
  month_colname=month_col,
  proportion_seasonal_years_threshold=threshold_proportion_seasonal_years
)

# Compute the duration block; there are normal warnings when it's only 0-es for seasonality:
# for those admin units without any seasonality, the duration of the block will be 'infinite')
check_pattern_seasonality <- paste("^SEASONALITY", toupper(type_of_seasonality), "[0-9]+_MTH$", sep = "_")
seasonality_source_dt <- seasonality_source_dt[, .SD, .SDcols = c(admin_id_col, grep(check_pattern_seasonality, names(seasonality_source_dt), value = TRUE))]

## Result file

**Long format, until further notice, is not saved.

In [ ]:
seasonality_long_dt <- melt(
  seasonality_source_dt,
  id.vars = grep(check_pattern_seasonality, names(seasonality_source_dt), value = TRUE, invert = TRUE), # all cols which don't follow the pattern
  variable.name = 'MONTH_BLOCK_SIZE',
  value.name =seasonality_col
  )

In [ ]:
seasonality_long_dt[, MONTH_BLOCK_SIZE := possible_month_block_sizes[match(MONTH_BLOCK_SIZE, grep(check_pattern_seasonality, names(seasonality_source_dt), value = TRUE))]]

# add remaining admin unit columns and save the final results
admin_seasonality_long_dt <- merge.data.table(admin_data, seasonality_long_dt, by = c(admin_id_col), all = TRUE)

In [ ]:
# order the columns
specific_cols <- setdiff(names(admin_seasonality_long_dt), names(admin_data)) # last columns
admin_seasonality_long_dt <- admin_seasonality_long_dt[, .SD, .SDcols = c(common_cols, specific_cols)]

In [ ]:
# Let's keep this code for now.
# # filename_admin_seasonality_long_dt <- paste(COUNTRY_CODE, gsub("\\.", "", as.character(threshold_for_seasonality)), type_of_seasonality, 'seasonality_long.csv', sep = '_')
# filename_admin_seasonality_long_dt <- paste(COUNTRY_CODE, type_of_seasonality, 'seasonality_long.csv', sep = '_')
# fwrite(admin_seasonality_long_dt, file.path(OUTPUT_DATA_PATH, filename_admin_seasonality_long_dt))

### Transform to wide format

In [ ]:
seasonality_wide_dt <- compute_min_seasonality_block(
    input_dt=seasonality_source_dt,
    seasonality_column_pattern=check_pattern_seasonality,
    vector_of_possible_month_block_sizes=possible_month_block_sizes,
    indicator=toupper(type_of_seasonality),
    seasonal_blocksize_colname=season_duration_col,
    valid_value = 1
)

In [ ]:
# Create a new, overall column 'SEASONALITY_' based on the values of columns in 'check_pattern_seasonality'
# seasonality_wide_dt <- seasonality_wide_dt[, (paste('SEASONALITY', toupper(type_of_seasonality), sep="_")) := ifelse(rowSums(.SD == 1) > 0, 1, 0), .SDcols = grep(check_pattern_seasonality, names(seasonality_source_dt), value = TRUE)]
seasonality_wide_dt <- seasonality_wide_dt[, (seasonality_col) := ifelse(rowSums(.SD == 1) > 0, 1, 0), .SDcols = grep(check_pattern_seasonality, names(seasonality_source_dt), value = TRUE)]
seasonality_wide_dt <- seasonality_wide_dt[, (grep(check_pattern_seasonality, names(seasonality_wide_dt), value = TRUE)) := NULL] # drop the columns with the pattern

In [ ]:
# add remaining admin unit columns and save the final results
admin_seasonality_wide_dt <- merge.data.table(admin_data, seasonality_wide_dt, by = c(admin_id_col), all = TRUE)
admin_seasonality_wide_dt <- admin_seasonality_wide_dt[, .SD, .SDcols = c(common_cols, seasonality_cols)]
head(admin_seasonality_wide_dt)

**Save output**

In [ ]:
# Create the filename
filename_seasonality_wide_csv <- file.path(OUTPUT_DATA_PATH, paste(COUNTRY_CODE, type_of_seasonality, 'seasonality.csv', sep = '_'))
filename_seasonality_wide_parquet <- file.path(OUTPUT_DATA_PATH, paste(COUNTRY_CODE, type_of_seasonality, 'seasonality.parquet', sep = '_'))

fwrite(admin_seasonality_wide_dt, filename_seasonality_wide_csv)
write_parquet(admin_seasonality_wide_dt, filename_seasonality_wide_parquet)

log_msg(paste0("Precipitation seasonality results saved under: ", filename_seasonality_wide_csv))